In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys
cwd = os.getcwd()

api_path = os.path.join(cwd, '..', 'tools')
models_2d_path = os.path.join(cwd, '..', 'models_25gaussians', 'jensen_minimax')
models_5d_path = os.path.join(cwd, '..', 'models_5d_gaussians', 'jensen_minimax')
sys.path.append(api_path)

In [ ]:
import numpy as np
import sklearn.datasets
import time
import random
import glob
from tqdm import tqdm

import pyro
from pyro.infer import MCMC, NUTS, HMC
from functools import partial
import ot

from matplotlib import pyplot as plt

import torch, torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable
from torch import autograd
from torch.distributions import MultivariateNormal, Normal, Independent, Uniform

from utils import (Evolution,
                   prepare_25gaussian_data, 
                   prepare_swissroll_data,
                   prepare_gaussians,
                   prepare_train_batches,
                   prepare_dataloader,
                   logging)
from gan_fc_models import (Generator_fc, 
                           Discriminator_fc, 
                           weights_init_1, 
                           weights_init_2)
from visualization import (send_file_to_remote,
                           sample_fake_data,
                           plot_fake_data_mode,
                           plot_fake_data_projection,
                           plot_discriminator_2d,
                           plot_potential_energy,
                           mh_sampling_plot_2d,
                           langevin_sampling_plot_2d,
                           mala_sampling_plot_2d,
                           xtry_langevin_sampling_plot_2d,
                           epoch_visualization)
from mh_sampling import mh_sampling
from ebm_sampling import (langevin_sampling, 
                          mala_sampling,
                          xtry_langevin_sampling,
                          gan_energy,
                          Gaussian_mixture,
                          IndependentNormal,
                          DotDict)

In [ ]:
class GaussianMixture():
    def __init__(self, **kwargs):
        self.n = 25
        self.d = 2
        self.sigma = kwargs.get('sigma', 0.05)
        self.means = np.array(list(itertools.product(np.arange(-2,3), repeat=2)))
        self.torch_dist = torch.distributions.Normal(torch.FloatTensor(self.means), torch.FloatTensor([self.sigma]))
        
    def sample(self, n_pts):
        pass

    def pdf(self, x):
        assert len(x.shape) == 2
        x = x.reshape(-1, 2)
        x = torch.FloatTensor(x)
        assert x.shape[1] == self.d
        pis = self.torch_dist.log_prob(x.unsqueeze(1).repeat(1, len(self.means), 1)).sum(2).exp().sum(1) / float(len(self.means))
        return pis.detach().cpu().numpy()

In [ ]:
@torch.no_grad()
def get_pis_estimate(X_gen, target, n_pts=4000, sample_method='grid', density_method='gmm', sigma=0.05):
    gm = GaussianMixture(sigma=sigma)
    gm = target
    target_pdf = lambda x: gm.log_prob(x).exp().detach().cpu().numpy()

    if density_method == 'kde':
        G_ker = gaussian_kde(X_gen.detach().cpu().numpy().reshape(2, -1))
        pi_g_f = lambda x: G_ker.pdf(x.reshape(2, -1))

    elif density_method == 'gmm':
        gm_g = mixture.GaussianMixture(n_components=25)
        gm_g.fit(X_gen.detach())
        pi_g_f = lambda x: np.exp(gm_g.score_samples(x))
    optD = lambda x: target_pdf(x) / (target_pdf + pi_g_f(x) + 1e-8)

    if sample_method == 'grid':
        n_pts = int(n_pts**.5)
        X = np.mgrid[-3:3:4./n_pts, -3:3:4./n_pts].reshape(2, -1).T
    elif sample_method == 'mc':
        pass
        #X = 4*torch.rand(n_pts, 2) - 2.

    pi_d = target_pdf(X) #gm.pdf(X)
    pi_g = pi_g_f(X)

    opt_ds = optD(X)
    opt_ds[pi_d < 1e-9] = 0.

    return pi_d, pi_g, opt_ds, X


class Evolution(object):
    def __init__(self, means, sigma=0.05, scaler=None):
        self.means = means
        self.sigma = sigma
        self.scaler = scaler
        self.mode_std = []
        self.high_quality_rate = []
        self.jsd = []

        self.kl_pis = []
        self.js_pis = []
        self.l2_div = []

    @staticmethod
    def make_assignment(X_gen, means, sigma=0.05):
        n_modes, x_dim = means.shape
        dists = torch.norm((X_gen[:, None, :] - means[None, :, :]), p=2, dim=-1)
        assignment = dists < 4 * sigma
        return assignment

    @staticmethod
    def compute_mode_std(X_gen, assignment):
        """
        X_gen(torch.FloatTensor) - (n_pts, x_dim)
        
        """
        x_dim = X_gen.shape[-1]
        n_modes = assignment.shape[1]
        std = 0
        for mode_id in range(n_modes):
            xs = X_gen[assignment[:, mode_id]]
            if xs.shape[0] > 1:
                std_ = (1 / (2**(x_dim - 1) * (xs.shape[0] - 1)) * ((xs - xs.mean(0))**2).sum())**.5
                std += std_
        std /= n_modes
        return std

    @staticmethod
    def compute_high_quality_rate(assignment):
        high_quality_rate = assignment.max(1)[0].sum() / float(assignment.shape[0])
        return high_quality_rate

    @staticmethod
    def compute_jsd(assignment):
        n_modes = assignment.shape[1]
        assign_ = torch.cat([assignment, torch.zeros(assignment.shape[0]).unsqueeze(1)], -1)
        assign_[:, -1][assignment.sum(1) == 0] = 1
        sample_dist = assign_.sum(dim=0) / float(assign_.shape[0])
        sample_dist /= sample_dist.sum()
        uniform_dist = torch.FloatTensor([1. / n_modes for _ in range(n_modes)] + [0]).to(assignment.device)
        M = .5 * (uniform_dist + sample_dist)
        JSD = .5 * (sample_dist * torch.log((sample_dist + 1e-7) / M)).sum() + .5 * (uniform_dist * torch.log((uniform_dist + 1e-7) / M)).sum()

        return JSD

    def compute_emd(self):
        pass

    def invoke(self, X_gen, D=None, compute_discrim_div=False):
        assignment = Evolution.make_assignment(X_gen, self.means, self.sigma)
        mode_std = Evolution.compute_mode_std(X_gen, assignment)
        self.mode_std.append(mode_std.item())
        h_q_r = Evolution.compute_high_quality_rate(assignment)
        self.high_quality_rate.append(h_q_r.item())
        jsd = Evolution.compute_jsd(assignment)
        self.jsd.append(jsd.item())

        pi_d, pi_g, opt_ds, X = get_pis_estimate(X_gen, n_pts=4000, sample_method='grid', density_method='gmm', sigma=self.sigma)
        kl = pi_g * (np.log(pi_g) - np.log(pi_d + 1e-10))
        kl[pi_g == 0.] = 0.
        kl = np.mean(kl).item()
        self.kl_pis.append(kl)

        m = .5 * (pi_g + pi_d)
        js = .5 * (pi_g * (np.log(pi_g) - np.log(m)) + pi_d * (np.log(pi_d) - np.log(m)))
        js[(pi_g == 0.) + (pi_d == 0.)] = 0.
        js = np.mean(js).item()
        self.js_pis.append(js)

        if self.scaler is not None and compute_discrim_div and D is not None:
            ds = D(torch.FloatTensor(self.scaler.transform(X)))[:, 0]
            div = torch.abs(ds - torch.FloatTensor(opt_ds))
            inf_dist = torch.max(div).item()
            l2_dist = ((div**2).mean()).item()**.5
            self.l2_div.append(l2_dist)

    def as_dict(self):
        d = dict(mode_std=self.mode_std, 
                hqr=self.high_quality_rate,
                jsd=self.jsd,
                kl_pis=self.kl_pis,
                js_pis=self.js_pis,
                l2_div=self.l2_div)
        return d

In [ ]:
dim = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

target_args = DotDict()
target_args.device = device
target_args.num_gauss = 25
n_col = 5
n_row = target_args.num_gauss // n_col
s = 1
u = torch.ones(dim)
###create points
sigma = 0.05
coef_gaussian = 1./target_args.num_gauss
target_args.p_gaussians = [torch.tensor(coef_gaussian)]*target_args.num_gauss
locs = [torch.tensor([(i-2)*s,(j-2)*s] + [0]* (dim - 2)).to(device) 
        for i in range(n_col) for j in range(n_row)]
target_args.locs = locs
target_args.covs = [(sigma**2)*torch.eye(dim).to(device)]*target_args.num_gauss
target_args.dim = dim
true_target = Gaussian_mixture(target_args).log_prob


n_dim = G.n_dim
loc_proposal = torch.zeros(n_dim).to(G.device)
scale_proposal = torch.ones(n_dim).to(G.device)
proposal_args = DotDict()
proposal_args.device = device
proposal_args.loc = loc_proposal
proposal_args.scale = scale_proposal
proposal = IndependentNormal(proposal_args)

In [ ]:
true_target_energy = Gaussian_mixture(target_args).energy

x_range = 3.0
y_range = 3.0
norm_grads = False
plot_potential_energy(true_target_energy,
                      x_range,
                      y_range,
                      device,
                      norm_grads = norm_grads,
                      num_points = 100)

In [ ]:
grad_step = 1e-4
eps_scale = (grad_step * 2) ** 0.5
n_steps = 100
n_batches = 10
langevin_sampling_plot_2d(true_target,
                          proposal,
                          X_train,  
                          batch_size_sample = batch_size_sample,
                          grad_step = grad_step,
                          eps_scale = eps_scale,
                          n_steps = n_steps,
                          n_batches = n_batches)